TM-Score

In [1]:
import re
import os
import subprocess

def create_item(query, reference):    
    if not os.path.isfile('TMalign'):
        print(subprocess.getoutput('g++ -O3 -ffast-math -lm -o TMalign {s}'.format(s='TMalign.cpp')))

    batcmd = f'./TMalign {query} {reference}'
    try:
        result = subprocess.check_output(batcmd, shell=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running TMalign: {e}")
    
    score_lines = []
    for line in result.decode().split("\n"):
        if line.startswith("TM-score"):
            score_lines.append(line)

    # Fetch the chain number
    key_getter = lambda s: re.findall(r"Chain_[12]{1}", s)[0]
    score_getter = lambda s: float(re.findall(r"=\s+([0-9.]+)", s)[0])
    results_dict = {key_getter(s): score_getter(s) for s in score_lines}

    return results_dict

query = "../data/new_pdb.pdb"
reference = "../data/Filled_pdb.pdb"
print(create_item(query, reference))

{'Chain_1': 0.53699, 'Chain_2': 0.18099}


LDDT metric (Local Distance Difference Test)

In [2]:
from Bio.PDB import PDBParser
import numpy as np
from scipy.spatial import cKDTree

def parse_ca_coords(pdb_path):
    """Return sorted list of residue IDs and corresponding Cα coords."""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('S', pdb_path)
    coords = []
    res_ids = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    coords.append(residue['CA'].get_coord())
                    res_ids.append((chain.id, residue.id[1]))
    return np.array(coords), res_ids

def compute_lddt(ref_pdb, pred_pdb, cutoff=15.0):
    # load coordinates and residue mappings
    ref_coords, ref_ids = parse_ca_coords(ref_pdb)
    pred_coords, pred_ids = parse_ca_coords(pred_pdb)
    assert ref_ids == pred_ids, "Residue ordering/mapping must match"

    # build a neighbor-search tree on the reference coords
    tree = cKDTree(ref_coords)
    thresholds = [0.5, 1.0, 2.0, 4.0]
    per_residue_scores = []

    for i, r_coord in enumerate(ref_coords):
        # neighbors within cutoff (excluding self)
        neighbors = [j for j in tree.query_ball_point(r_coord, cutoff) if j != i]
        if not neighbors:
            per_residue_scores.append(np.nan)
            continue

        # distance arrays
        d_ref  = np.linalg.norm(ref_coords[neighbors]  - r_coord, axis=1)
        d_pred = np.linalg.norm(pred_coords[neighbors] - pred_coords[i], axis=1)
        diffs  = np.abs(d_pred - d_ref)

        # fraction under each threshold
        fracs = [(diffs < t).sum() / len(diffs) for t in thresholds]
        per_residue_scores.append(np.mean(fracs))

    # global LDDT
    global_lddt = np.nanmean(per_residue_scores)
    return per_residue_scores, global_lddt

if __name__ == '__main__':
    ref   = 'experimental.pdb'
    pred  = 'alphafold_model.pdb'
    per_res, g_lddt = compute_lddt(ref, pred)
    print(f"Global LDDT = {g_lddt*100:.2f}")
    # e.g. print per‐residue if you like
    # for idx, score in enumerate(per_res):
    #     print(f"Res {idx+1:4d}: LDDT = {score*100:5.1f}")

FileNotFoundError: [Errno 2] No such file or directory: 'experimental.pdb'

In [10]:
"""lDDT protein distance score."""
# import jax.numpy as jnp
import numpy as jnp


def lddt(predicted_points,
         true_points,
         true_points_mask,
         cutoff=15.,
         per_residue=False):
  """Measure (approximate) lDDT for a batch of coordinates.

  lDDT reference:
  Mariani, V., Biasini, M., Barbato, A. & Schwede, T. lDDT: A local
  superposition-free score for comparing protein structures and models using
  distance difference tests. Bioinformatics 29, 2722–2728 (2013).

  lDDT is a measure of the difference between the true distance matrix and the
  distance matrix of the predicted points.  The difference is computed only on
  points closer than cutoff *in the true structure*.

  This function does not compute the exact lDDT value that the original paper
  describes because it does not include terms for physical feasibility
  (e.g. bond length violations). Therefore this is only an approximate
  lDDT score.

  Args:
    predicted_points: (batch, length, 3) array of predicted 3D points
    true_points: (batch, length, 3) array of true 3D points
    true_points_mask: (batch, length, 1) binary-valued float array.  This mask
      should be 1 for points that exist in the true points.
    cutoff: Maximum distance for a pair of points to be included
    per_residue: If true, return score for each residue.  Note that the overall
      lDDT is not exactly the mean of the per_residue lDDT's because some
      residues have more contacts than others.

  Returns:
    An (approximate, see above) lDDT score in the range 0-1.
  """

  assert len(predicted_points.shape) == 3
  assert predicted_points.shape[-1] == 3
  assert true_points_mask.shape[-1] == 1
  assert len(true_points_mask.shape) == 3

  # Compute true and predicted distance matrices.
  dmat_true = jnp.sqrt(1e-10 + jnp.sum(
      (true_points[:, :, None] - true_points[:, None, :])**2, axis=-1))

  dmat_predicted = jnp.sqrt(1e-10 + jnp.sum(
      (predicted_points[:, :, None] -
       predicted_points[:, None, :])**2, axis=-1))

  dists_to_score = (
      (dmat_true < cutoff).astype(jnp.float32) * true_points_mask *
      jnp.transpose(true_points_mask, [0, 2, 1]) *
      (1. - jnp.eye(dmat_true.shape[1]))  # Exclude self-interaction.
  )

  # Shift unscored distances to be far away.
  dist_l1 = jnp.abs(dmat_true - dmat_predicted)

  # True lDDT uses a number of fixed bins.
  # We ignore the physical plausibility correction to lDDT, though.
  score = 0.25 * ((dist_l1 < 0.5).astype(jnp.float32) +
                  (dist_l1 < 1.0).astype(jnp.float32) +
                  (dist_l1 < 2.0).astype(jnp.float32) +
                  (dist_l1 < 4.0).astype(jnp.float32))

  # Normalize over the appropriate axes.
  reduce_axes = (-1,) if per_residue else (-2, -1)
  norm = 1. / (1e-10 + jnp.sum(dists_to_score, axis=reduce_axes))
  score = norm * (1e-10 + jnp.sum(dists_to_score * score, axis=reduce_axes))

  return score[0]


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from Bio.PDB import *

def get_coordinates(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure('protein', pdb_file)
    coordinates = []
    
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.get_name() == 'CA':  # Only consider alpha carbons
                        coordinates.append(atom.get_coord())
    
    return np.array([coordinates])

true_points = get_coordinates("1ubi.pdb")
predicted_points = get_coordinates("1ubi_shifted.pdb")
true_points_mask = np.ones(shape=[1, 76, 1])
lddt(predicted_points, true_points, true_points_mask)


1.0

In [12]:
true_points = get_coordinates("1ubi.pdb")
predicted_points = get_coordinates("unnamed.pdb")
true_points_mask = np.ones(shape=[1, 76, 1])
lddt(predicted_points, true_points, true_points_mask)

0.9791666666666674